In [ ]:
import torch
import numpy as np
from data.imagedata import img_attr_label
from model.resnet50 import CB
from PIL import Image
import pandas as pd
import torchvision
import os
import matplotlib.pyplot as plt
from tabulate import tabulate

In [ ]:
device='cuda'
imagedir='/media/lijia/系统/data/train_align'
imagepath='n000002/0001_01.jpg'
maadpath='/media/lijia/系统/data/vggface2/MAAD_Face.csv'
idpath='train_id.csv'

In [ ]:
target_attributions= ['Male','Young', 'Middle_Aged', 'Senior','Asian','White','Black','Rosy_Cheeks','Shiny_Skin','Bald','Wavy_Hair','Receding_Hairline','Bangs','Sideburns',
                'Black_Hair','Blond_Hair','Brown_Hair','Gray_Hair','No_Beard','Mustache',
                '5_o_Clock_Shadow','Goatee','Oval_Face','Square_Face','Round_Face','Double_Chin',
                'High_Cheekbones','Chubby','Obstructed_Forehead', 'Fully_Visible_Forehead','Brown_Eyes',
                'Bags_Under_Eyes','Bushy_Eyebrows','Arched_Eyebrows','Mouth_Closed','Smiling',
                'Big_Lips','Big_Nose','Pointy_Nose','Heavy_Makeup','Wearing_Hat','Wearing_Earrings',
                'Wearing_Necktie','Wearing_Lipstick','No_Eyewear','Eyeglasses','Attractive']

In [ ]:
maad=pd.read_csv(maadpath)
id_csv=pd.read_csv(idpath)

In [ ]:
# load model
model=CB(len(target_attributions),8631)
model.load_state_dict(torch.load('/home/lijia/codes/202210/cbw-face/checkpoints/13_cb.pth.tar')['state_dict'])
model.to(device)
model.eval()

In [ ]:
def read_image(dir,path):
    img=Image.open(os.path.join(dir,path))
    img=torchvision.transforms.Resize(224)(img)
    img=np.array(img)
    img = img[:, :, ::-1]  # RGB -> BGR
    img = img.astype(np.float32)
    img -=np.array([91.4953, 103.8827, 131.0912])
    img = img.transpose(2, 0, 1)  # C x H x W
    img = torch.from_numpy(img).float()
    return img

In [ ]:
image=read_image(imagedir,imagepath)

In [ ]:
attr=maad[maad['Filename']==imagepath][target_attributions]
id=id_csv[id_csv['Filename']==imagepath]['id']

In [ ]:
attr=np.array(attr).squeeze(0)
attr[attr==-1]=0
id=np.array(id).squeeze(0)

In [ ]:
image=image.to(device)
c,y=model(image.unsqueeze(0))

In [ ]:
c_label=c.cpu().detach().numpy().squeeze(0)
y_label=np.argmax(y.cpu().detach().numpy())

In [ ]:
print(type(target_attributions),type(attr.tolist()),type(c_label.tolist()))
table_c={
    'type\\attr_name':target_attributions,
    'GT':attr.tolist(),
    'predict':c_label.tolist()
}

print(tabulate(table_c,headers='keys'))

In [ ]:
def plt_attr(attr,concept):
    plt.figure(figsize=(20,30))
    font={
        'weight':'normal',
        'size':20,
    }
    plt.xlabel('attribution',font)
    plt.plot(attr.tolist(),target_attributions,linewidth=3,marker='o',markersize=10)
    plt.plot(concept.tolist(),target_attributions,linewidth=3,marker='o',markersize=10)
    # plt.xticks(np.arange(len(target_attributions)),target_attributions)
    plt.yticks(size=15)
    plt.legend(['GT','Prediction'])
    plt.show()

In [ ]:
plt_attr(attr,c_label)

## visual counterfactual
change the concept class in cbm and observing the change in identity

In [ ]:
def get_attrindex(an):
    return target_attributions.index(an)

In [ ]:
attr_name='Mustache'
classifier=model.get_f()

In [ ]:
cc=c.clone().detach()

origin=cc[0][get_attrindex(attr_name)].clone().detach()
print('origin:',origin)
after=1-origin
print('after:',after)
print('origin identity:{}'.format(y_label))
cc[0][get_attrindex(attr_name)]=after
y_modify=classifier(cc)
y_modify=np.argmax(y_modify.cpu().detach().numpy())
print('when change {} : {} to {},the identity is {}'.format(attr_name,origin,after,y_modify))